<a href="https://colab.research.google.com/github/Loga19818eeanvesh/Natural_Language_Processing-Assignments/blob/main/NLP_C2W4_L2_Word2Vec_CBOW_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install emoji

import sys
!{sys.executable} -m pip install emoji # to resolve: ModuleNotFoundError: No module named 'emoji'

import re
import nltk
from nltk.tokenize import word_tokenize
import emoji
import numpy as np

nltk.download('punkt')  # download pre-trained Punkt tokenizer for English

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# Define the 'relu' function that will include the steps previously seen
def relu(z):
    result = z.copy()
    result[result < 0] = 0
    return result

In [6]:
# Define the 'softmax' function that will include the steps previously seen
def softmax(z):
    e_z = np.exp(z)
    sum_e_z = np.sum(e_z)
    return e_z / sum_e_z

In [7]:
def get_dict(words):
  words = list(set(words))
  words.sort()
  l = len(words)

  ind = 0
  word2ind = {}
  ind2word = {}
  for w in words:
    word2ind[w] = ind
    ind2word[ind] = w
    ind += 1

  return word2ind, ind2word

# Define the 'tokenize' function that will include the steps previously seen
def tokenize(corpus):
    data = re.sub(r'[,!?;-]+', '.', corpus)
    data = nltk.word_tokenize(data)  # tokenize string to words
    data = [ ch.lower() for ch in data
             if ch.isalpha()
             or ch == '.'
             or emoji.get_emoji_regexp().search(ch)
           ]
    return data
    
# Define the 'get_windows' function as seen in a previous notebook
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1

# Define the 'word_to_one_hot_vector' function as seen in a previous notebook
def word_to_one_hot_vector(word, word2Ind, V):
    one_hot_vector = np.zeros(V)
    one_hot_vector[word2Ind[word]] = 1
    return one_hot_vector

# Define the 'context_words_to_vector' function as seen in a previous notebook
def context_words_to_vector(context_words, word2Ind, V):
    context_words_vectors = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]
    context_words_vectors = np.mean(context_words_vectors, axis=0)
    return context_words_vectors

# Define the generator function 'get_training_example' as seen in a previous notebook
def get_training_example(words, C, word2Ind, V):
    for context_words, center_word in get_windows(words, C):
        yield context_words_to_vector(context_words, word2Ind, V), word_to_one_hot_vector(center_word, word2Ind, V)

In [23]:
corpus = 'I am happy because I am learning'
words = tokenize(corpus)
word2Ind, Ind2word = get_dict(words)
V = len(word2Ind)
N=3

In [12]:
np.random.seed(10)


In [24]:
X = np.zeros((V,1))
X

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [25]:
W1 = np.random.rand(N,V)
print(W1)
b1 = np.random.rand(N,1)
print(b1)

[[0.30306256 0.24207588 0.55757819 0.56550702 0.47513225]
 [0.29279798 0.06425106 0.97881915 0.33970784 0.49504863]
 [0.97708073 0.44077382 0.31827281 0.51979699 0.57813643]]
[[0.85393375]
 [0.06809727]
 [0.46453081]]


In [28]:
W2 = np.random.rand(V,N)
print(W2)
b2 = np.random.rand(V,1)
print(b2)

[[0.46978529 0.59825567 0.14762019]
 [0.18403482 0.64507213 0.04862801]
 [0.24861251 0.54240852 0.22677334]
 [0.38141153 0.92223279 0.92535687]
 [0.56674992 0.53347088 0.01486002]]
[[0.97789926]
 [0.5730289 ]
 [0.791757  ]
 [0.56155736]
 [0.87733524]]


In [29]:
# BEGIN your code here
print(f'V (vocabulary size): {V}')
print(f'N (embedding size / size of the hidden layer): {N}')
print(f'size of W1: {W1.shape} (NxV)')
print(f'size of b1: {b1.shape} (Nx1)')
print(f'size of W2: {W2.shape} (VxN)')
print(f'size of b2: {b2.shape} (Vx1)')
# END your code here

V (vocabulary size): 5
N (embedding size / size of the hidden layer): 3
size of W1: (3, 5) (NxV)
size of b1: (3, 1) (Nx1)
size of W2: (5, 3) (VxN)
size of b2: (5, 1) (Vx1)


In [30]:
training_examples = get_training_example(words, 2, word2Ind, V)

In [31]:
x_array, y_array = next(training_examples)

In [32]:
x_array

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

In [33]:
y_array

array([0., 0., 1., 0., 0.])

In [34]:
x = x_array.copy()
x.shape = (V, 1)
print('x')
print(x)
print()

y = y_array.copy()
y.shape = (V, 1)
print('y')
print(y)

x
[[0.25]
 [0.25]
 [0.  ]
 [0.5 ]
 [0.  ]]

y
[[0.]
 [0.]
 [1.]
 [0.]
 [0.]]


In [36]:
z1 = np.dot(W1, x) + b1
z1

array([[1.27297187],
       [0.32721345],
       [1.07889294]])

In [37]:
h = relu(z1)
h

array([[1.27297187],
       [0.32721345],
       [1.07889294]])

In [38]:
# BEGIN your code here
z2 = np.dot(W2, h) + b2
# END your code here

z2

array([[1.93094641],
       [1.07084075],
       [1.53038125],
       [2.34721149],
       [1.78938318]])

In [39]:
# BEGIN your code here
y_hat = softmax(z2)
# END your code here

y_hat

array([[0.22334656],
       [0.09450181],
       [0.14962909],
       [0.33865766],
       [0.19386488]])

In [41]:
def cross_entropy_loss(y_predicted, y_actual):
    # BEGIN your code here
    loss = np.sum(-np.log(y_predicted)*y_actual)
    # END your code here
    return loss

In [42]:
cross_entropy_loss(y_hat, y)

1.899595784663097

In [43]:
# BEGIN your code here
grad_b2 = y_hat - y
# END your code here

grad_b2

array([[ 0.22334656],
       [ 0.09450181],
       [-0.85037091],
       [ 0.33865766],
       [ 0.19386488]])

In [44]:
# BEGIN your code here
grad_W2 = np.dot(y_hat - y, h.T)
# END your code here

grad_W2

array([[ 0.28431389,  0.073082  ,  0.24096703],
       [ 0.12029815,  0.03092226,  0.10195734],
       [-1.08249825, -0.2782528 , -0.91745917],
       [ 0.43110167,  0.11081334,  0.36537535],
       [ 0.24678454,  0.0634352 ,  0.20915945]])

In [45]:
# BEGIN your code here
grad_b1 = relu(np.dot(W2.T, y_hat - y))
# END your code here

grad_b1


array([[0.14994455],
       [0.14907287],
       [0.16098447]])

In [46]:
# BEGIN your code here
grad_W1 = np.dot(relu(np.dot(W2.T, y_hat - y)), x.T)
# END your code here

grad_W1

array([[0.03748614, 0.03748614, 0.        , 0.07497228, 0.        ],
       [0.03726822, 0.03726822, 0.        , 0.07453644, 0.        ],
       [0.04024612, 0.04024612, 0.        , 0.08049223, 0.        ]])

In [47]:
# BEGIN your code here
print(f'V (vocabulary size): {V}')
print(f'N (embedding size / size of the hidden layer): {N}')
print(f'size of grad_W1: {grad_W1.shape} (NxV)')
print(f'size of grad_b1: {grad_b1.shape} (Nx1)')
print(f'size of grad_W2: {grad_W2.shape} (VxN)')
print(f'size of grad_b2: {grad_b2.shape} (Vx1)')
# END your code here

V (vocabulary size): 5
N (embedding size / size of the hidden layer): 3
size of grad_W1: (3, 5) (NxV)
size of grad_b1: (3, 1) (Nx1)
size of grad_W2: (5, 3) (VxN)
size of grad_b2: (5, 1) (Vx1)


In [48]:
alpha = 0.03

W1_new = W1 - alpha * grad_W1
W2_new = W2 - alpha * grad_W2
b1_new = b1 - alpha * grad_b1
b2_new = b2 - alpha * grad_b2

In [49]:
W1

array([[0.30306256, 0.24207588, 0.55757819, 0.56550702, 0.47513225],
       [0.29279798, 0.06425106, 0.97881915, 0.33970784, 0.49504863],
       [0.97708073, 0.44077382, 0.31827281, 0.51979699, 0.57813643]])

In [50]:
# loop through each word of the vocabulary
for word in word2Ind:
    # extract the column corresponding to the index of the word in the vocabulary
    word_embedding_vector = W1[:, word2Ind[word]]
    
    print(f'{word}: {word_embedding_vector}')

am: [0.30306256 0.29279798 0.97708073]
because: [0.24207588 0.06425106 0.44077382]
happy: [0.55757819 0.97881915 0.31827281]
i: [0.56550702 0.33970784 0.51979699]
learning: [0.47513225 0.49504863 0.57813643]


In [51]:
W2.T

array([[0.46978529, 0.18403482, 0.24861251, 0.38141153, 0.56674992],
       [0.59825567, 0.64507213, 0.54240852, 0.92223279, 0.53347088],
       [0.14762019, 0.04862801, 0.22677334, 0.92535687, 0.01486002]])

In [52]:
# loop through each word of the vocabulary
for word in word2Ind:
    # extract the column corresponding to the index of the word in the vocabulary
    word_embedding_vector = W2.T[:, word2Ind[word]]
    
    print(f'{word}: {word_embedding_vector}')

am: [0.46978529 0.59825567 0.14762019]
because: [0.18403482 0.64507213 0.04862801]
happy: [0.24861251 0.54240852 0.22677334]
i: [0.38141153 0.92223279 0.92535687]
learning: [0.56674992 0.53347088 0.01486002]


In [54]:
# BEGIN your code here
W3 = (W1+W2.T)/2
# END your code here

W3

array([[0.38642393, 0.21305535, 0.40309535, 0.47345928, 0.52094109],
       [0.44552682, 0.35466159, 0.76061383, 0.63097032, 0.51425976],
       [0.56235046, 0.24470092, 0.27252307, 0.72257693, 0.29649823]])

In [55]:
# loop through each word of the vocabulary
for word in word2Ind:
    # extract the column corresponding to the index of the word in the vocabulary
    word_embedding_vector = W3[:, word2Ind[word]]
    
    print(f'{word}: {word_embedding_vector}')

am: [0.38642393 0.44552682 0.56235046]
because: [0.21305535 0.35466159 0.24470092]
happy: [0.40309535 0.76061383 0.27252307]
i: [0.47345928 0.63097032 0.72257693]
learning: [0.52094109 0.51425976 0.29649823]
